In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import training dataframe
train_data = pd.read_csv('../input/paddy-disease-classification/train.csv')
train_data.head()

In [3]:
# Get unique label names
labels = list(train_data['label'].unique())
labels

In [4]:
plt.figure(figsize = (10,5))
sns.histplot(data=train_data, x='label')
plt.xticks(rotation=90)
plt.show()

## Data Augmentation

In [5]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.45),
    tf.keras.layers.RandomFlip('horizontal')
])

train_data= tf.keras.utils.image_dataset_from_directory("../input/paddy-disease-classification/train_images",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=labels,
                                                        color_mode='rgb',
                                                        image_size=(400,400),
                                                        shuffle=True,
                                                        validation_split=0.1,
                                                        subset='training',
                                                        seed=42)

validation_data= tf.keras.utils.image_dataset_from_directory("../input/paddy-disease-classification/train_images",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=labels,
                                                        color_mode='rgb',
                                                        image_size=(400,400),
                                                        shuffle=True,
                                                        validation_split=0.1,
                                                        subset='validation',
                                                        seed=42)

## Import EfficientNetB2 model

In [6]:
# Import pre-trained model
pre_trained_model = tf.keras.applications.EfficientNetB2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(400,400,3)
)


# Design model
inputs = tf.keras.Input(shape=(400,400,3))
x = data_augmentation(inputs)
x = pre_trained_model(x, training=False)
x = tf.keras.layers.Dense(512,activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs,output)

In [7]:
model.summary()

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2, restore_best_weights=True)

In [9]:
for layers in pre_trained_model.layers:
    layers.trainable = False

In [10]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [11]:
history=model.fit(
    train_data,
    validation_data=validation_data,
    epochs=15,
    callbacks=[callback]
)
